In [2]:
import torch

ModuleNotFoundError: No module named 'torch'

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import sklearn as skl
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

# PART A

# prepare data
df = pd.read_csv("data.csv")
x1 = df['x1']
x2 = df['x2']
y = df['outcome']
groups = df['group']

# set up model
model = LogisticRegression()
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# train and test model
def train_and_test(X, y):
    accuracies = []
    disparities = []

    print(f"{'Fold':<10} | {'Accuracy':<12} | {'Demographic Disparity':<20}")
    print("-" * 50)

    for i, (train_index, val_index) in enumerate(kf.split(X)):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        group_val = groups.iloc[val_index]
        
        model.fit(X_train, y_train)
        
        y_pred = model.predict(X_val)
        
        # Calculate Accuracy
        acc = accuracy_score(y_val, y_pred)
        accuracies.append(acc)
        
        prob_g0 = y_pred[group_val == 0].mean()
        prob_g1 = y_pred[group_val == 1].mean()
        
        disparity = abs(prob_g0 - prob_g1)
        disparities.append(disparity)
        
        print(f"{i+1:<10} | {acc:<12.4f} | {disparity:<20.4f}")

    print("-" * 40)
    print(f"{'AVERAGE':<10} | {np.mean(accuracies):<12.4f} | {np.mean(disparities):<20.4f}")


train_and_test(pd.concat([x1, x2], axis=1), y)

print("=" * 50)
# PART B
corr_x1 = df['x1'].corr(df['group'])
corr_x2 = df['x2'].corr(df['group'])
print(f"Correlation between x1 and group: {corr_x1}")
print(f"Correlation between x2 and group: {corr_x2}")

train_and_test(x1.to_frame(), y)
print("=" * 50)

#PART C
# discontinuities when on decision boundary?
# sigmoid function on disparity term
# \frac{1}{1+e^{-x}}


# PART D
def sigmoid(x, reg=1):
    return reg * (1 / (1 + np.exp(-x)))

accuracies_list = []
disparities_list = []

for reg in range(0.0, 1.05, 0.05):
    
    sigmoid(x, reg)
    acc, disp = 0, 0 # fix this
    if reg == 0:
        print("Disparity 0: accuracy is", disp)
    accuracies_list.append(acc)
    disparities_list.append(disp)

plt.plot(disparities_list, accuracies_list)
plt.xlabel('Demographic Disparity')
plt.ylabel('Accuracy')
plt.title('Accuracy-Disparity Tradeoff')
plt.grid(True)
plt.show()

#minimize Logistic Regression loss function (log loss) and add disparity term
#output weights 
def fairness_loss(model, X, y, A, lam):
    